In [3]:
import cv2
import numpy as np

# Load the YOLO model
net = cv2.dnn.readNet("/yolov3.weights", "/yolov3.cfg")

# Load the classes file
classes = []
with open("/labels.name", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Define the colors for the bounding boxes
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Set the input size for the model
width = 416
height = 416

# Set the confidence threshold for object detection
confidence_threshold = 0.5

# Set the non-max suppression threshold
nms_threshold = 0.4

# Load the video or image
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the video or image
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to a blob
    blob = cv2.dnn.blobFromImage(frame, 1/255, (width, height), swapRB=True, crop=False)

    # Set the input blob for the model
    net.setInput(blob)

    # Run the model
    outputs = net.forward(net.getUnconnectedOutLayersNames())

    # Lists to store detected bounding boxes, confidences, and class IDs
    boxes = []
    confidences = []
    class_ids = []

    # Loop through the detections
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > confidence_threshold:
                # Get the bounding box coordinates
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                w = int(detection[2] * frame.shape[1])
                h = int(detection[3] * frame.shape[0])

                # Calculate the top-left corner of the bounding box
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Add the box, confidence, and class ID to their respective lists
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-Maximum Suppression (NMS)
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)

    # Loop through the remaining boxes after NMS
    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i]
            color = colors[class_ids[i]]

            # Draw the bounding box
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

            # Label with class name and confidence
            label = f"{classes[class_ids[i]]}: {confidences[i]:.2f}"
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the output
    cv2.imshow("Object Detection", frame)

    # Exit on key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture
cap.release()
cv2.destroyAllWindows()
